In [1]:
import numpy as np
import math as m
import random
from copy import deepcopy

In [2]:
def indipop(populasi,p_kromos):
    gen = []
    for i in range(populasi):
        temp = []
        for i in range(p_kromos):
            temp.append(random.randint(0,1))
        gen.append(temp)
    return gen

In [3]:
def fungsi(x1,x2):
    f = (4-2.1*m.pow(x1,2)+m.pow(x1,4)/3)*m.pow(x1,2)+(x1*x2)+(-4+4*m.pow(x2,2))*m.pow(x2,2)
    return f

In [4]:
def chromoX(b,N,garray):
    total, sigma = 0, 0
    for i in range(N//2):
        total = total + garray[i]*2**-(i+1)
        sigma = sigma + 2**-(i+1)
    c = b[0] + ((b[1] - b[0])/sigma)*total
    return c

def chromoY(b,N,garray):
    total, sigma = 0, 0
    g = garray[N//2:]
    for i in range(N//2):
        total = total + g[i]*2**-(i+1)
        sigma = sigma + 2**-(i+1)
    c = b[0] + ((b[1] - b[0])/sigma)*total
    return c

In [5]:
def minFitness(x1,x2):
    fitness = -(fungsi(x1,x2))
    return fitness

In [6]:
def elitism(gen,fit):
    sort = sorted(zip(fit,gen), reverse=True)
    elitgen = []
    elitfit = []
    for i in range(2):
        elitgen.append(sort[i][1])
        elitfit.append(sort[i][0])
    return elitgen, elitfit

In [7]:
def tournament(gen,x1,x2):
    best = []
    N = len(gen)
    for i in range(N):
        k = i-1
        bst = -1
        for j in range(3):
            r = random.randint(0,N-1)
            if bst == -1 or minFitness(x1[r],x2[r])>minFitness(x1[bst],x2[bst]):
                bst = r
        if not best or best[k]!=gen[bst]:
            best.append(gen[bst])
        else:
            while(best[k]==gen[bst]):
                r = random.randint(0,N-1)
                if bst != r:
                    bst = r
            best.append(gen[bst])
    return best

In [8]:
def crossover(psel,kromos,pc):
    r = np.random.uniform(0,1)
    if r<= pc:
        n1 = np.random.randint(0,(kromos//2)-1)
        n2 = np.random.randint((kromos//2),kromos-1)
        
        j = 0
        child = []
        while(j<len(psel)):
            psel1 = deepcopy(psel[j])
            psel2 = deepcopy(psel[j+1])

            for i in range(kromos):
                if i>=n1 and i<=n2:
                    psel1[i],psel2[i] = deepcopy(psel2[i]), deepcopy(psel1[i])
            child.append(psel1)
            child.append(psel2)
            j += 2
        return child
    else:
        return psel

In [9]:
def mutasi(childm):
    length = len(childm)
    lm = 0.1
    for i in range(length):
        r = np.random.uniform(0,1)
        if r<=lm :
            if childm[i] == 1:
                childm[i] = 0
            else:
                childm[i] = 1
    return childm

In [104]:
#MAIN PROGRAM

#INISIALISASI
pop = 50
kromos = 30
pmut = 1/pop
pcross = 0.7
generasi = 250
batas1 = [-3,3]
batas2 = [-2,2]
x = indipop(pop,kromos)

#MULAI ITERASI GENERASI
for i in range(generasi):
    #print()
    #print('Gen-',i+1)
    
    #Mencari xgen(x1) dan ygen(x2) untuk populasi(x) 
    xgen = []
    ygen = []
    for i in range(pop):
        xgen.append(chromoX(batas1,kromos,x[i]))
        ygen.append(chromoY(batas2,kromos,x[i]))

    #Mencari Nilai Fitness dari populasi
    fit = []
    for i in range(pop):
        fit.append(minFitness(xgen[i],ygen[i]))

    #Elitism 2 individu terbaik
    eg = []
    ef = []
    eg, ef = elitism(x,fit)
    elit = sorted(zip(ef,eg), reverse=True)
    
    #print()
    #print(*elit, sep = "\n")
    #print()

    #Parent Selection
    psel = []
    psel = tournament(x,xgen,ygen)

    #Crossover
    child = []
    child = crossover(psel,kromos,pcross)

    #Mutasi
    for i in range(len(child)):
        r = np.random.uniform(0,1)
        if r<=pmut:
            child[i] = mutasi(child[i])

    #Mencari fitness untuk child
    fchild = []
    for i in range(len(child)):
        fchild.append(minFitness(chromoX(batas1,kromos,child[i]),chromoY(batas2,kromos,child[i])))

    #Menentukan populasi untuk generasi selanjutnya
    x = []
    f = []
    x = child + eg
    f = fchild + ef
    bestgen = sorted(zip(f,x), reverse=True)[:pop]

    #Mendapat populasi baru untuk generasi selanjutnya 
    x = []
    for i in range(pop):
        x.append(bestgen[i][1])

    #print(*bestgen, sep = "\n")
    #print()
    #print(*x, sep = "\n")
    
X1 = chromoX(batas1,kromos,bestgen[0][1])
X2 = chromoY(batas2,kromos,bestgen[0][1])
print()
print('Best Fitness dan Individu:')
print(bestgen[0][0],' - ',bestgen[0][1])
print()
print('X1     : ',X1)
print('X2     : ',X2)
print('Fungsi : ',fungsi(X1,X2))


Best Fitness dan Individu:
1.0310852152574297  -  [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]

X1     :  -0.09073152867213974
X2     :  0.7207861568041016
Fungsi :  -1.0310852152574297
